### Extract data from SEC website using Selenium Package

In [2]:
import os
import time
import shutil
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configuration variables
CIK = "0000789019"
companyName = "Microsoft"
downloadDir = "D:\\pythonProject\\DCF\\Data"
tempDownloadDir = "D:\\pythonProject\\DCF\\Data\\Temp"
maxFilings = 10

sicCode = None
sicDesc = None
location = None
ticker = None
sharePrice = 421.53 # Will be automated later

# Set up WebDriver
chromeOptions = webdriver.ChromeOptions()
chromeOptions.add_experimental_option("prefs", {'download.default_directory': tempDownloadDir})
service = Service('D:/pythonProject/DCF/Scrapping/chromedriver.exe')
driver = webdriver.Chrome(service=service, options=chromeOptions)


def wait_for_element(driver, by, value, timeout=10):
    return WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((by, value)))

def extract_company_info():
    global sicCode, sicDesc, location, ticker
    try:
        # Expand the Company Information section
        companyInfo = wait_for_element(driver, By.ID, "entityInformationHeader")
        driver.execute_script("arguments[0].click();", companyInfo)

        # Extract SIC code and description
        sicElement = wait_for_element(driver, By.ID, "sicSection")
        sicCode = sicElement.find_element(By.XPATH, ".//span[@id='SIC']/a").text
        sicDesc = sicElement.find_element(By.ID, "SICDescription").text.strip(" - ")
        print(f"SIC Code: {sicCode}, Description: {sicDesc}")
        
        # Extract Location
        locElement = wait_for_element(driver, By.ID, "stateLocationSection")
        location = locElement.find_element(By.ID, "stateLocation").text
        print(f"Location: {location}")

        # Extract Ticker
        tickerElement = wait_for_element(driver, By.ID, "ticker")
        ticker = tickerElement.text.split()[0]
        print(f"Ticker: {ticker}")

    except Exception as e:
        print(f"An error occurred while extracting company information: {e}")

def process_filing(row, index):
    try:
        filingDate = row.find_element(By.XPATH, ".//td[3]").text
        filingLink = row.find_element(By.XPATH, ".//a[contains(@href, 'index.htm')]")
        filingURL = filingLink.get_attribute('href')

        # Open the filing in a new tab
        driver.execute_script("window.open(arguments[0], '_blank');", filingURL)
        driver.switch_to.window(driver.window_handles[-1])

        # Interact with the interactive data and download the Excel file
        wait_for_element(driver, By.ID, "interactiveDataBtn").click()
        wait_for_element(driver, By.LINK_TEXT, "View Excel Document").click()

        # Allow time for the file to download
        time.sleep(10)

        # Verify the download and move the file
        listFiles = os.listdir(tempDownloadDir)
        if not listFiles:
            raise FileNotFoundError("No files found in the temporary download directory.")

        latestFile = max([os.path.join(tempDownloadDir, file) for file in listFiles], key=os.path.getctime)
        newFileName = os.path.join(downloadDir, f"SEC_{companyName} {filingDate}.xlsx")
        shutil.move(latestFile, newFileName)
        print(f"File has been downloaded and renamed to {newFileName}")

    except Exception as e:
        print(f"An error occurred during processing filing {index + 1}: {e}")
    finally:
        # Close the tab and return to the main tab
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

def main():
    try:
        driver.get(f"https://www.sec.gov/edgar/browse/?CIK={CIK}")

        # Extract and print company information (including SIC code)
        extract_company_info()

        wait_for_element(driver, By.ID, "btnViewAllFilings").click()
        wait_for_element(driver, By.ID, "searchbox").send_keys("10-K")

        for i in range(maxFilings):
            tenkRows = driver.find_elements(By.XPATH, "//td[normalize-space(text())='10-K']/parent::tr")
            
            if i >= len(tenkRows):
                print("No more '10-K' filings found.")
                break

            process_filing(tenkRows[i], i)

            # Refresh the search to process the next filing
            wait_for_element(driver, By.ID, "searchbox").clear()
            wait_for_element(driver, By.ID, "searchbox").send_keys("10-K")
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "filingsTable")))

    finally:
        driver.quit()

if __name__ == "__main__":
    main()
    print(f"SIC Description: {sicDesc}")
    print(f"Share Price: {sharePrice}")


SIC Code: 7372, Description: Services-Prepackaged Software
Location: WA
Ticker: MSFT
File has been downloaded and renamed to D:\pythonProject\DCF\Data\SEC_Microsoft 2024-07-30.xlsx
File has been downloaded and renamed to D:\pythonProject\DCF\Data\SEC_Microsoft 2023-07-27.xlsx
File has been downloaded and renamed to D:\pythonProject\DCF\Data\SEC_Microsoft 2022-07-28.xlsx
File has been downloaded and renamed to D:\pythonProject\DCF\Data\SEC_Microsoft 2021-07-29.xlsx
File has been downloaded and renamed to D:\pythonProject\DCF\Data\SEC_Microsoft 2020-07-30.xlsx
File has been downloaded and renamed to D:\pythonProject\DCF\Data\SEC_Microsoft 2019-08-01.xlsx
File has been downloaded and renamed to D:\pythonProject\DCF\Data\SEC_Microsoft 2018-08-03.xlsx
No more '10-K' filings found.
SIC Description: Services-Prepackaged Software
Share Price: 421.53


#### Extract data from SEC Excel using Pandas package

In [38]:
import json
import os
import pandas as pd
from dateutil.parser import parse
import re

def parse_date(key, value):
    try:
        return parse(key).strftime('%b. %d, %Y')
    except (ValueError, TypeError):
        pass
    try:
        return parse(value).strftime('%b. %d, %Y')
    except (ValueError, TypeError):
        pass
    return None

def find_attr(value, attrNames): 
    value = value.lower()
    res = None
    for attr in attrNames:
        if attr == value:
            return attr.capitalize(), False
    for attr in attrNames:
        if attr in value:
            res = attr.capitalize()
    return res, True

# Insert newline every time a number ends and is followed by a word
def insert_newlines(text):
    return re.sub(r'(\d+)\s+(?=[a-zA-Z])', r'\1\n', text)

def extract_lease_info(filePath):
    try:
        # Load the Excel file to inspect sheet names
        xls = pd.ExcelFile(filePath)

        # Find the sheet that contains "leases (tables)" in its name
        sheetName = next((sheet for sheet in xls.sheet_names if "leases (tables)" in sheet.lower()), None)
        if not sheetName:
            print("No sheet found containing 'leases (tables)'.")
            return None, None
        
        print(f"Found Leases Tables Sheet: {sheetName}")
        df = pd.read_excel(filePath, sheet_name=sheetName)

        opLease = None
        maturities = {}
        maturityKeywords = ["maturities of lease liabilities", "maturity", "maturities"]

        for index, row in df.iterrows():
            # Convert row to string, replace '\n' with space, and lowercase it for uniformity
            # Insert newlines after numbers followed by a word
            strRow = insert_newlines(' '.join(row.astype(str)).replace('\n', ' ').lower())

            if not opLease:
                # Match the pattern with or without a space after the "$" and extract the full number with commas
                match = re.search(r'operating lease cost.*?\$\s*([\d,]+)', strRow, re.IGNORECASE)
                if match:
                    opLease = int(match.group(1).replace(',', ''))
            
            if any(keyword in strRow for keyword in maturityKeywords):
                nextRow = [token for token in strRow.split() if token != '$']
                
                for i in range(len(nextRow) - 1):
                    # Check if the current token is a year and the next token is a dollar amount
                    if re.match(r'\d{4}', nextRow[i]):
                        try:
                            year = nextRow[i]
                            maturities[year] = int(nextRow[i + 1].replace(',', ''))
                        except ValueError:
                            print(f"Skipping invalid value: '{nextRow[i + 1]}' for year {year}")
        return opLease, maturities

    except Exception as e:
        print(f"Error while extracting lease information: {e}")
        return None, None

def processSheet(filePath, sheetIds, attrNames, dataMap):
    # Load the Excel file
    xls = pd.ExcelFile(filePath)

    # Find the first sheet that matches any of the specified identifiers
    sheetName = next((sheet for sheet in xls.sheet_names if any(sheetId in sheet.lower() for sheetId in sheetIds)), None)
    if not sheetName:
        print(f"No sheet found containing any of the identifiers: {sheetIds}")
        return dataMap
    
    df = pd.read_excel(filePath, sheet_name=sheetName)
    print(f"Data from sheet: {sheetName}")

    dataKey = None
    innerMap = {}
    matched = {}

    # Convert the DataFrame to JSON
    dfJson = df.to_json(orient='records')
    data = json.loads(dfJson)
    
    # Iterate over each row in the DataFrame to find dates in column B
    for i, row in enumerate(data):
        items = list(row.items())
        prev_attr = ""

        for j, (key, value) in enumerate(items):
            if j == 1 and not dataKey:
                dataKey = parse_date(key, value)
            if j == 0 and value:
                matched_attr, partial = find_attr(value, attrNames)
                if matched_attr:
                    prev_attr = matched_attr
            if j == 1 and prev_attr:
                if prev_attr not in innerMap or (matched[prev_attr] == True and partial == False):
                    innerMap[prev_attr] = value
                    matched[prev_attr] = partial
        if dataKey:
            if dataKey not in dataMap:
                dataMap[dataKey] = {}
            dataMap[dataKey].update(innerMap)
    return dataMap

def processAllFiles(companyName, directory):
    files = [f for f in os.listdir(directory) if f.startswith(f"SEC_{companyName}")]
    dataMap = {}

    # Define possible sheet identifiers and attributes for each statement
    sheetIds = {
        'balanceSheet': ['balance sheet', 'consolidated balance sheets'],
        'incomeStatement': ['income statements', 'consolidated statements of oper', 'statements of operations'],
        'cash_flow': ['cash flows', 'consolidated statements of cash', 'statements of cash fl']
    }
    attrNames = {
        'balanceSheet': ["cash", "cash and cash equivalents", "goodwill", "total current assets", "marketable securities", 
                         "short-term investments", "total assets", "short-term debt", "current portion of long-term debt", "long-term debt", 
                         "total current liabilities", "total liabilities", "retained earnings", "total stockholders’ equity"],
        'incomeStatement': ["revenue", "other revenue", "cost of revenue", "gross margin", "gross profit", "selling, general and administrative",
                            "sales and marketing", "general and administrative", "general & administrative expenses", "selling expenses", 
                            "research and development", "interest expense", "operating income", "income before taxes", "income before income taxes", 
                            "net loss before income tax", "(benefit from) provision for income taxes", "provision for income taxes", "net income"],
        'cash_flow': ["depreciation, amortization, and other", "net cash from operations", "net cash used in by operating activities", 
                      "net cash provided by operating activities", "net cash used in financing", "net cash from (used in) financing", 
                      "net cash provided by (used in) financing activities", "net cash provided by financing activities", 
                      "additions to property and equipment", "acquisition", "net cash used in investing"]
    }

    for file in files:
        filePath = os.path.join(directory, file)
        print(f"Processing file: {filePath}")

        dataMap = processSheet(filePath, sheetIds['balanceSheet'], attrNames['balanceSheet'], dataMap)
        dataMap = processSheet(filePath, sheetIds['incomeStatement'], attrNames['incomeStatement'], dataMap)
        dataMap = processSheet(filePath, sheetIds['cash_flow'], attrNames['cash_flow'], dataMap)
        dataMap = processSheet(filePath, ['components of other income'], ['interest expense'], dataMap)
        dataMap = processSheet(filePath, ['balance sheets (parenthetical)'], ['common stock, outstanding'], dataMap)

        # Extract lease information
        opLease, maturities = extract_lease_info(filePath)
        if opLease or maturities:
            latest = list(dataMap.keys())[-1]
            innerMap = {}
            if opLease:
                innerMap["Operating Lease Cost"] = opLease
            if maturities:
                innerMap["Lease Maturities"] = maturities
            if innerMap:
                dataMap[latest].update(innerMap)
    return dataMap

# Specify the company name and directory to search
companyName = "Microsoft"
directory = "D:\\pythonProject\\DCF\\Data"

# Process all files matching the pattern
dataMap = processAllFiles(companyName, directory)

# Output the results
for key, value in dataMap.items():
    print(f"{key}:")
    if isinstance(value, dict):
        for sub_key, sub_value in value.items():
            print(f"  {sub_key}: {sub_value}")
    else:
        print(f"  {value}")


Processing file: D:\pythonProject\DCF\Data\SEC_Microsoft 2018-08-03.xlsx
Data from sheet: BALANCE SHEETS
Data from sheet: INCOME STATEMENTS
Data from sheet: CASH FLOWS STATEMENTS
Data from sheet: Components of Other Income (Exp
Data from sheet: BALANCE SHEETS (Parenthetical)
Found Leases Tables Sheet: LEASES (Tables)
Processing file: D:\pythonProject\DCF\Data\SEC_Microsoft 2019-08-01.xlsx
Data from sheet: BALANCE SHEETS
Data from sheet: INCOME STATEMENTS
Data from sheet: CASH FLOWS STATEMENTS
Data from sheet: Components of Other Income (Exp
Data from sheet: BALANCE SHEETS (Parenthetical)
Found Leases Tables Sheet: LEASES (Tables)
Processing file: D:\pythonProject\DCF\Data\SEC_Microsoft 2020-07-30.xlsx
Data from sheet: BALANCE SHEETS
Data from sheet: INCOME STATEMENTS
Data from sheet: CASH FLOWS STATEMENTS
Data from sheet: Components of Other Income (Exp
Data from sheet: BALANCE SHEETS (Parenthetical)
Found Leases Tables Sheet: LEASES (Tables)
Processing file: D:\pythonProject\DCF\Data\

#### Calculate additional variables & Operating Lease and R&D capitalization

In [45]:
from sentence_transformers import SentenceTransformer, util

def extractExcel(filePath, sheetName):
    try:
        df = pd.read_excel(filePath, sheet_name=sheetName)
        dfJson = df.to_json(orient='records')
        data = json.loads(dfJson)
        return data

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to find the most similar description based on sentence embeddings
def find_most_similar_description(sicDesc, data):
    most_similar = None
    max_similarity = -1
    
    # Compute the embedding for the SIC description
    sic_embedding = model.encode(sicDesc.lower(), convert_to_tensor=True)
    keywords = sicDesc.lower().split()
    
    for entry in data:
        for key, value in entry.items():
            if isinstance(value, str):
                entry_lower = value.lower()
                entry_embedding = model.encode(entry_lower, convert_to_tensor=True)
                
                # Calculate the cosine similarity between the SIC description and the entry description
                similarity = util.pytorch_cos_sim(sic_embedding, entry_embedding).item()
                
                # Boost similarity score based on keyword matches
                keyword_boost = sum(1 for word in keywords if word in entry_lower) * 0.1
                similarity += keyword_boost
                
                # Update the most similar entry if the similarity is higher
                if similarity > max_similarity:
                    max_similarity = similarity
                    most_similar = {
                        "entry": entry,
                        "similarity_score": similarity
                    }
    
    return most_similar

def calc_tax_rate(data):
    incomeBeforeTax = (data.get('Income before income taxes', 0) + data.get('Income before taxes', 0) 
                       - data.get('Net loss before income tax', 0))
    return data.get('Provision for income taxes', 0) / incomeBeforeTax if incomeBeforeTax else 0

def find_bond_rating(data, similarDesc, rev, intCoverage):
    finance = ["Banking", "Banking (Canadian)", "Banking (Foreign)", "Banking (Midwest)", "Financial Services", "Insurance (Diversified)",
                "Insurance (Life)", "Insurance (Property/Casualty)", "Investment Companies (Domestic)", "Investment Companies (Foreign)",
                "Investment Companies (Income)", "Securities Brokerage", "Thrift Institutions"]
    category = "For large non-financial service firms"
    nextRows = None

    if rev < 5000:
        category = "For smaller and riskier firms"
    if similarDesc in finance:
        category = "For financial service firms (default spreads are slighty different)"
    
    for index, row in enumerate(data):
        if row.get('Inputs for synthetic rating estimation') == category:
            nextRows = data[index + 3:index + 18] 

    for row in nextRows:
        items = list(row.items())
        if intCoverage > items[0][1] and intCoverage <= items[1][1]:
            return items[2][1], items[3][1]     
    return "", 0

def calc_rd(similarDesc, dataMap, dates):
    rdValList = []
    rdVal = 0
    capitalized = 0.0
    currAmor = 0

    if similarDesc:
        amorPeriod = int(similarDesc["entry"].get("Amortization Period"))
        for date in reversed(dates):
            if 'Research and development' in dataMap[date]:
                rdValList.append((date, dataMap[date]['Research and development']))
            if len(rdValList) > amorPeriod:
                break
        if rdValList:
            first = rdValList[0][1]
            portion = 1.0 / amorPeriod
            if len(rdValList) > 1:
                rdVal = sum(value / amorPeriod for _, value in rdValList[1:])
                capitalized = sum(float(value) * (1.0 - (portion * index)) for index, (_, value) in enumerate(rdValList[1:], start=1))
                capitalized += float(first)
                currAmor = rdVal
                rdVal = first - rdVal
            else:
                rdVal = first
                capitalized = float(first)
    else:
        print("No similar entry found.")
    
    return rdVal, capitalized, currAmor 

def calc_deprec_leased_assets(data, preTaxCostDebt):
    commitments = data.get("Lease Maturities", 0)
    pv = [0] * 6
    avg = 0
    leaseDebt = 0
    
    for index, value in enumerate(commitments.values()):
        if index < 5:
            pv[index] = value / ((1 + preTaxCostDebt)**(index + 1))
            leaseDebt += pv[index]
            avg += value
        else:
            pv[5] += value

    cnt = round(pv[5] / (avg / 5))
    if cnt > 0 and pv[5] > 0:
        pv[5] /= cnt
        pv[5] = (pv[5] * (1 - (1 + preTaxCostDebt) ** (-cnt)) / preTaxCostDebt) / ((1 + preTaxCostDebt)**(index + 1))
    leaseDebt += pv[5]
    
    return leaseDebt / (cnt + 5), leaseDebt


def find_singular_fin_metrics(data, date, dataMap):
    amortization  = extractExcel("D:\\pythonProject\\DCF\\Data\\Core\\R&DConv.xlsx", "Amortizable Lives Look-up Table")
    dataBond = extractExcel("D:\\pythonProject\\DCF\\Data\\Core\\ratings.xlsx", "Start here Ratings sheet")

    rev = data.get('Revenue', 0) + data.get('Other revenue', 0)
    debt = data.get('Short-term debt', 0) + data.get('Long-term debt', 0) + data.get("Current portion of long-term debt", 0)

    # Find the most similar description in the data using Sentence Transformers
    similarDesc = find_most_similar_description(str(sicDesc), amortization)
    rdVal, capRd, currAmor = calc_rd(similarDesc, dataMap, dates)

    taxRate = calc_tax_rate(data)
    intCoverage = debt / (data.get('Interest expense', 0) * -1)
    govBondRate = 0.0384
    bondRating, defaultSpread = find_bond_rating(dataBond, similarDesc["entry"]["Industry Name"], rev, intCoverage)
    preTaxCostDebt = govBondRate + defaultSpread
    
    # Find depreciation on leased assets
    deprecLeased, capLease = calc_deprec_leased_assets(data, preTaxCostDebt)
    opLease = data.get("Operating Lease Cost", 0) - deprecLeased

    capEx = (data.get("Additions to property and equipment", 0) * -1)
    deprec = data.get("Depreciation, amortization, and other", 0)

    # Working Capital
    wc = data.get("Total current assets", 0) - data.get("Total current liabilities", 0) 
    nonCashWC = wc - data.get("Cash and cash equivalents", 0)

    # Adjusted EBIT
    ebit = data.get("Operating income", 0)

    # NOPAT
    nopat = ebit * (1 - taxRate)

    # Adjusted Interest Expense
    adjIntExpense = (data.get("Interest expense", 0) * -1) + (preTaxCostDebt * capLease)
    
    innerMap = {
        "Total revenue" : rev,
        "Total debt" : debt,
        "Tax rate" : taxRate,
        "Interest coverage" : intCoverage,
        "Government bond rate" : govBondRate,
        "Default spread" : defaultSpread,
        "Bond rating" : bondRating,
        "Pre-tax cost of debt" : preTaxCostDebt,
        "Gross margin" : (rev - data.get('Cost of revenue', 0)) / rev,
        "Value of R&D asset" : capRd,
        "Amortization of R&D asset for current year" : currAmor,
        "R&D value" : rdVal,
        "Depreciation on leased asset" : deprecLeased,
        "Capitalized leases" : capLease,
        "Operating lease value" : opLease,
        "Capital expenditures" : capEx,
        "Adjusted CapEx" : capEx + data.get("Research and development", 0) + data.get("Operating Lease Cost", 0),
        "Depreciation & amortization" : deprec,
        "Adjusted Depreciation" : deprec + deprecLeased + currAmor,
        "Working capital" : wc,
        "Non-cash working capital" : nonCashWC,
        "Non-operating assets" : data.get("Cash and cash equivalents", 0) + data.get("Short-term investments", 0),
        "EBIT" : ebit,
        "NOPAT" : nopat,
        "Adjusted interest expense" : adjIntExpense
    }
    dataMap[date].update(innerMap)


def calc_rev_growth(rev1, rev2):
    return (rev1 - rev2) / abs(rev2) if rev2 else 0

def calc_marginal_tax(latest, prev):
    taxRate = calc_tax_rate(latest)
    prevTaxRate = calc_tax_rate(prev)
    taxableInc = latest.get('Provision for income taxes', 0) / taxRate
    prevTaxableInc = prev.get('Provision for income taxes', 0) / prevTaxRate
    return (latest.get('Provision for income taxes', 0) - prev.get('Provision for income taxes', 0)) / (taxableInc - prevTaxableInc)

def find_compound_fin_metrics(latest, prev, date, dataMap):
    rdVal = latest.get("R&D value", 0)
    margTaxRate = calc_marginal_tax(latest, prev)
    
    adjEbit = latest.get("EBIT", 0) + latest.get("Operating lease value", 0) 
    adjNopat = adjEbit * (1 - latest.get("Tax rate", 0))
    
    adjEbit = adjEbit + + rdVal + (rdVal * margTaxRate)
    adjNopat = adjNopat + rdVal + (rdVal * margTaxRate)
    adjNetInc = latest.get("Net income", 0) + rdVal + (rdVal * margTaxRate)
    
    changeWC = latest.get("Working capital", 0) - prev.get("Working capital", 0)
    if changeWC < 0:
        changeWC = (latest.get("Total revenue", 0) - prev.get("Total revenue", 0)) * (latest.get("Working capital", 0) / latest.get("Total revenue", 0))
    
    changeNonCashWC = latest.get("Non-cash working capital", 0) - prev.get("Non-cash working capital", 0)
    if changeNonCashWC < 0:
        changeNonCashWC = (latest.get("Total revenue", 0) - prev.get("Total revenue", 0)) * (latest.get("Non-cash working capital", 0) / latest.get("Total revenue", 0))
    
    investedCap = latest.get("Total assets", 0) - latest.get("Total current liabilities", 0) - latest.get("Non-operating assets", 0)
    adjInvCap = investedCap + prev.get("Capitalized leases", 0) + latest.get("Value of R&D asset", 0)
    
    innerMap = {
        "Revenue growth rate" : calc_rev_growth(latest.get("Total revenue", 0), prev.get("Total revenue", 0)),
        "Marginal tax rate" : margTaxRate,
        "Adjusted EBIT" : adjEbit,
        "Adjusted NOPAT" : adjNopat,
        "Adjusted net income" : adjNetInc,
        "Change in working capital" : changeWC,
        "Change in non-cash working capital" : changeNonCashWC,
        "EBIT margin" : adjEbit / latest.get("Total revenue", 0),
        "Net margin" : adjNetInc / latest.get("Total revenue", 0),
        "Invested capital" : investedCap,
        "Adjusted invested capital" : adjInvCap,
        "Sales to capital" : latest.get("Total revenue", 0) / adjInvCap,
    }
    dataMap[date].update(innerMap)


data = extractExcel("D:\\pythonProject\\DCF\\Data\\Core\\R&DConv.xlsx", "Amortizable Lives Look-up Table")
dataBond = extractExcel("D:\\pythonProject\\DCF\\Data\\Core\\ratings.xlsx", "Start here Ratings sheet")

# Initialize the Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Date indexes for dataMap
dates = list(dataMap.keys())


for i in range(len(dates)):
    find_singular_fin_metrics(dataMap[dates[i]], dates[i], dataMap)
    if i != 0:
        find_compound_fin_metrics(dataMap[dates[i]], dataMap[dates[i - 1]], dates[i], dataMap)

# Output the results
for key, value in dataMap.items():
    print(f"{key}:")
    if isinstance(value, dict):
        for sub_key, sub_value in value.items():
            print(f"  {sub_key}: {sub_value}")
        print("\n")
    else:
        print(f"  {value}")

c:\Users\DrEm0\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\DrEm0\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\DrEm0\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\DrEm0\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header o

Jun. 30, 2018:
  Cash and cash equivalents: 11946
  Short-term investments: 121822
  Total current assets: 169662
  Goodwill: 35683
  Total assets: 258848
  Short-term debt: 0
  Current portion of long-term debt: 3998
  Total current liabilities: 58488
  Long-term debt: 72242
  Total liabilities: 176130
  Retained earnings: 13682
  Total stockholders’ equity: 82718
  Revenue: 110360
  Cost of revenue: 38353
  Gross margin: 0.652473722363175
  Research and development: 14726
  Sales and marketing: 17469
  General and administrative: 4754
  Operating income: 35058
  Income before income taxes: 36474
  Provision for income taxes: 19903
  Net income: 16571
  Net cash from operations: 43884
  Depreciation, amortization, and other: 10261
  Net cash from (used in) financing: -33590
  Additions to property and equipment: -11632
  Acquisition: -888
  Net cash used in investing: -6061
  Interest expense: -2733
  Common stock, outstanding: 7677000000.0
  Operating Lease Cost: 1585
  Lease Maturit

#### Calculate DCF 

In [61]:
import openpyxl
import xlwings as xw

# Function to update the Excel file using openpyxl
def update_excel_text(filePath, sheetName, cell, location):
    try:
        workbook = openpyxl.load_workbook(filePath)
        sheet = workbook[sheetName]
        sheet[cell] = location
        workbook.save(filePath)
    except Exception as e:
        print(f"An error occurred while updating the Excel file: {e}")

# Function to extract the calculated values using xlwings
def extract_excel_value(filePath, sheetName, cells):
    try:
        # Open the Excel file with xlwings in the background (without opening the Excel window)
        app = xw.App(visible=False)
        workbook = xw.Book(filePath)
        sheet = workbook.sheets[sheetName]

        # Extract the values from the specified cells
        res = []
        for cell in cells:
            value = sheet[cell].value  # Get the calculated value
            res.append(value if value is not None else 0)  # Handle None values

        # Close the workbook and quit the app
        workbook.close()
        app.quit()

        return res
    except Exception as e:
        print(f"An error occurred while extracting values from the Excel file: {e}")
        return []

# Main function to update the Excel sheet and then extract values
def calc_erp(location):
    states = ["AL", "AK", "AZ", "AR", "AS", "CA", "CO", "CT", "DE", "DC", "FL", "GA", "GU", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", 
              "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "MP", "OH", "OK", "OR", "PA", "PR", "RI", 
              "SC", "SD", "TN", "TX", "TT", "UT", "VT", "VA", "VI", "WA", "WV", "WI", "WY"]
    if location in states:
        location = "United States"

    # Update the location in the target cell (e.g., B2 in the "Country Lookup" sheet)
    update_excel_text("D:\\pythonProject\\DCF\\Data\\Core\\ctryprem.xlsx", "Country Lookup", "B2", location)
    
    # Extract the calculated values after updating the location
    crp, erp = extract_excel_value("D:\\pythonProject\\DCF\\Data\\Core\\ctryprem.xlsx", "Country Lookup", ["B10", "B11"])
    return crp, erp

def calc_singular_dcf_metrics(latest, date, dataMap):
    crp, erp = calc_erp(location)
    dataBetas = extractExcel("D:\pythonProject\DCF\Data\Core\indbetas.xlsx", "Global Industry averages")
    similarDesc = find_most_similar_description(str(sicDesc), dataBetas)
    
    debt = latest.get("Total debt", 0)
    taxRate = latest.get("Tax rate", 0)
    govBondRate = latest.get("Government bond rate", 0)
    preTaxCostDebt = latest.get("Pre-tax cost of debt", 0)
    avgDebtMaturity = 5

    marketStrDebt = (latest.get("Interest expense", 0) * -1) * (1 - (1 + preTaxCostDebt)**(-avgDebtMaturity)) / preTaxCostDebt + debt / ((1 + preTaxCostDebt)**avgDebtMaturity)
    
    capLease = latest.get("Capitalized leases", 0)
    totalDebt = marketStrDebt + capLease
    totalEq = sharePrice * (latest.get("Common stock, outstanding", 0) / 1000000)
    totalCapital = totalDebt + totalEq
    
    weightEq = totalEq / totalCapital
    weightDebt = totalDebt / totalCapital

    levBeta = similarDesc["entry"]["Unlevered Beta"] * (1 + (1 - taxRate) * (totalDebt / totalEq))
    costEq = govBondRate + levBeta * erp
    costDebt = preTaxCostDebt * (1 - taxRate)
    costCap = costEq * weightEq + costDebt * weightDebt
    
    debtRatio = 1 - (totalEq / (totalCapital + capLease))
    wcRev = latest.get("Non-cash working capital", 0) / latest.get("Total revenue", 0)

    innerMap = {
        "Total market equity" : totalEq,
        "Total market debt" : totalDebt,
        "Total capital" : totalCapital,
        "Equity risk premium" : erp,
        "Risk free rate" : govBondRate,
        "Unlevered beta for equity" : similarDesc["entry"]["Unlevered Beta"],
        "Levered beta for equity" : levBeta,
        "Debt Ratio" : debtRatio,
        "WC to revenue" : wcRev,
        "Cost of Equity" : costEq,
        "Cost of Debt" : costDebt,
        "Cost of Capital" : costCap,
    }
    dataMap[date].update(innerMap)

def calc_compound_dcf_metrics(latest, prev, date, dataMap):
    prevEq = prev.get("Total stockholders’ equity", 0)
    prevDebt = prev.get("Total debt", 0)
    
    capRd = latest.get("Value of R&D asset", 0)
    currAmor = latest.get("Amortization of R&D asset for current year", 0)
    capLease = latest.get("Capitalized leases", 0)
    
    totalBVEq = prevEq + capRd - latest.get("Research and development", 0) + currAmor
    totalBVDebt = prevDebt + capLease

    reinvRate = (latest.get("Adjusted CapEx", 0) - latest.get("Adjusted Depreciation", 0) + latest.get("Change in non-cash working capital", 0)) / latest.get("Adjusted NOPAT", 0)
    roic = latest.get("Adjusted NOPAT", 0) / latest.get("Adjusted invested capital", 0)

    innerMap = {
        "Total adjusted book equity" : totalBVEq,
        "Total adjusted book debt" : totalBVDebt,
        "ROE" : latest.get("Adjusted net income", 0) / totalBVEq,
        "ROIC" : roic,
        "Reinvestment rate" : reinvRate,
        "Expected growth" : reinvRate * roic,
    }
    dataMap[date].update(innerMap)

dates = list(dataMap.keys())

def project_fcff(data, yearsLen):
    dataBetas = extractExcel("D:\pythonProject\DCF\Data\Core\indbetas.xlsx", "Global Industry averages")
    similarDesc = find_most_similar_description(str(sicDesc), dataBetas)

    # Assumptions
    endGrowthRate = 0.04
    endBeta = 1.01
    endErp = 0.046
    endDebtRaio = similarDesc["entry"]["Market Debt/Capital"]
    endPreTaxCostDebt = similarDesc["entry"]["Pre-tax cost of debt"]
    endTaxRate = similarDesc["entry"]["Average effective tax rate"]
    endRoc = similarDesc["entry"]["After-tax ROC"]

    expGrowth = [0] * yearsLen
    cumGrowth = [0] * yearsLen
    reinvRate = [0] * yearsLen
    ebit = [0] * yearsLen
    taxRate = [0] * yearsLen
    nopat = [0] * yearsLen
    netCapEx = [0] * yearsLen
    netWC = [0] * yearsLen
    fcff = [0] * yearsLen
    costCap = [0] * yearsLen
    cumCostCap = [0] * yearsLen
    pv = [0] * yearsLen

    endCostEq = endBeta * endErp + data.get("Risk free rate", 0)
    endWeightEq = 1 - endDebtRaio
    endCostDebt = endPreTaxCostDebt * (1 - endTaxRate)
    endCostCap = endCostDebt * endDebtRaio + endCostEq * endWeightEq

    chgTaxRate = (data.get("Tax rate", 0) - endTaxRate) / yearsLen
    chgExpGrowth = (data.get("Expected growth", 0) - endGrowthRate) / (yearsLen / 2)
    chgReinvRate = (data.get("Reinvestment rate", 0) - (endGrowthRate / endRoc)) / (yearsLen / 2)
    chgCostCap = (data.get("Cost of Capital", 0) - endCostCap) / (yearsLen / 2)

    prev = 1
    tempCostCap = 1
    tempGrowth = 1

    for i in range(yearsLen):
        expGrowth[i] = data.get("Expected growth", 0) if i < 5 else expGrowth[i - 1] - chgExpGrowth
        reinvRate[i] = data.get("Reinvestment rate", 0) if i < 5 else reinvRate[i - 1] - chgReinvRate
        tempGrowth *= (1 + expGrowth[i])
        cumGrowth[i] = tempGrowth

        ebit[i] = data.get("Adjusted EBIT", 0) * cumGrowth[i]
        taxRate[i] = data.get("Tax rate", 0) - chgTaxRate if i == 0 else taxRate[i - 1] - chgTaxRate
        nopat[i] = ebit[i] * (1 - taxRate[i])
        netWC[i] = data.get("Total revenue", 0) * (cumGrowth[i] - prev) * data.get("WC to revenue", 0)
        netCapEx[i] = nopat[i] * reinvRate[i] - netWC[i]
        fcff[i] = nopat[i] - netCapEx[i] - netWC[i]

        costCap[i] = data.get("Cost of Capital", 0) if i < 5 else costCap[i - 1] - chgCostCap
        tempCostCap *= (1 + costCap[i])
        cumCostCap[i] = tempCostCap
        pv[i] = fcff[i] / cumCostCap[i]
        prev = cumGrowth[i]

    tvNopat = nopat[yearsLen - 1] / (1 - data.get("Marginal tax rate", 0)) * (1 - data.get("Tax rate", 0)) * (1 + endGrowthRate)
    tvWC = ((data.get("Total revenue", 0) * (1 + expGrowth[0])**yearsLen) - (data.get("Total revenue", 0) * (1 + endGrowthRate)**yearsLen)) * data.get("WC to revenue", 0)
    tvCapEx = (endGrowthRate / endRoc) * tvNopat - tvWC
    tvFCFF = tvNopat - tvCapEx - tvWC

    endVal = tvFCFF / (endCostCap - endGrowthRate)
    sumPV = sum(val for val in pv)
    pvTVFirm = endVal / cumCostCap[-1]
    valOpAssets = pvTVFirm + sumPV
    valCashNonOpAssets = data.get("Goodwill", 0) + data.get("Non-operating assets", 0)
    valFirm = valOpAssets + valCashNonOpAssets

    marketDebt = data.get("Total debt", 0) + data.get("Operating lease value", 0)
    marketEq = valFirm - marketDebt
    valEqShare = marketEq / (data.get("Common stock, outstanding", 0) / 1000000) 
    res = sharePrice / valEqShare - 1 
    
    return {
        "Expected growth rate" : expGrowth,
        "Cumulated growth" : cumGrowth,
        "Reinvestment rate" : reinvRate,
        "EBIT" : ebit,
        "Tax rate" : taxRate,
        "NOPAT" : nopat,
        "Net CapEx" : netCapEx,
        "Net WC" : netWC,
        "FCFF" : fcff,
        "Cost of capital" : costCap,
        "Cumulated cost of capital" : cumCostCap,
        "Present value" : pv,
        "Value at the end of growth" : endVal,
        "Present value of FCFF in high growth phase" : sumPV,
        "Present value of terminal value of firm" : pvTVFirm,
        "Value of operating assets of the firm" : valOpAssets,
        "Value of cash, marketable securities & non-operating assets" : valCashNonOpAssets,
        "Value of firm" : valFirm,
        "Market debt" : marketDebt,
        "Market equity" : marketEq,
        "Value of equity per share" : valEqShare,
        "Stock price" : sharePrice,
        "% under or over valued" : res,
    }


for i in range(len(dates)):
    calc_singular_dcf_metrics(dataMap[dates[i]], dates[i], dataMap)
    if i != 0:
        calc_compound_dcf_metrics(dataMap[dates[i]], dataMap[dates[i - 1]], dates[i], dataMap)

# Output the results
for key, value in dataMap.items():
    print(f"{key}:")
    if isinstance(value, dict):
        for sub_key, sub_value in value.items():
            print(f"  {sub_key}: {sub_value}")
        print("\n")
    else:
        print(f"  {value}")

print('\n')

res = project_fcff(dataMap[dates[i]], 10)

for key, value in res.items():
    print(f"{key}: {value}")



<>:54: SyntaxWarning: invalid escape sequence '\p'
<>:124: SyntaxWarning: invalid escape sequence '\p'
<>:54: SyntaxWarning: invalid escape sequence '\p'
<>:124: SyntaxWarning: invalid escape sequence '\p'
C:\Users\DrEm0\AppData\Local\Temp\ipykernel_11216\2325449017.py:54: SyntaxWarning: invalid escape sequence '\p'
  dataBetas = extractExcel("D:\pythonProject\DCF\Data\Core\indbetas.xlsx", "Global Industry averages")
C:\Users\DrEm0\AppData\Local\Temp\ipykernel_11216\2325449017.py:124: SyntaxWarning: invalid escape sequence '\p'
  dataBetas = extractExcel("D:\pythonProject\DCF\Data\Core\indbetas.xlsx", "Global Industry averages")


Jun. 30, 2018:
  Cash and cash equivalents: 11946
  Short-term investments: 121822
  Total current assets: 169662
  Goodwill: 35683
  Total assets: 258848
  Short-term debt: 0
  Current portion of long-term debt: 3998
  Total current liabilities: 58488
  Long-term debt: 72242
  Total liabilities: 176130
  Retained earnings: 13682
  Total stockholders’ equity: 82718
  Revenue: 110360
  Cost of revenue: 38353
  Gross margin: 0.652473722363175
  Research and development: 14726
  Sales and marketing: 17469
  General and administrative: 4754
  Operating income: 35058
  Income before income taxes: 36474
  Provision for income taxes: 19903
  Net income: 16571
  Net cash from operations: 43884
  Depreciation, amortization, and other: 10261
  Net cash from (used in) financing: -33590
  Additions to property and equipment: -11632
  Acquisition: -888
  Net cash used in investing: -6061
  Interest expense: -2733
  Common stock, outstanding: 7677000000.0
  Operating Lease Cost: 1585
  Lease Maturit

#### Plot results

In [ ]:
### --- Import Packages --- ###
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import datetime as dt
import scipy.stats
import sys
from IPython.display import display


ebitda, revenue, expense, adminExp, amortization, otherRev, otherExp = 0
ebit = 0
nopat, incomeTax = 0
capEx, chok, liquidExp = 0  
discountRate = 0
dcf, terminalVal = 0
equityVal, enterpriseVal = 0


from sentence_transformers import SentenceTransformer, util
import json